In [3]:
import pandas as pd
import requests
from pprint import pprint
import json


In [14]:
WEATHER_URL = "https://api.open-meteo.com/v1/forecast"
AIR_URL = "https://air-quality-api.open-meteo.com/v1/air-quality"

cities = [
    ("Lisbon", 38.7167, -9.1333),
    ("Thessaloniki", 40.6436, 22.9309),
    ("Brussels", 50.8505, 4.3488),
    ("Barcelona", 41.3888, 2.1590),
    ("Berlin", 52.5244, 13.4105),
]

rows = []

for name, lat, lon in cities:
    # Weather
    w_params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
        "timezone": "Europe/Athens",
    }
    w = requests.get(WEATHER_URL, params=w_params, timeout=30)
    w.raise_for_status()
    cw = w.json().get("current", {})

    # Air quality
    a_params = {
        "latitude": lat,
        "longitude": lon,
        "current": "european_aqi,pm2_5,nitrogen_dioxide,ozone",
        "timezone": "Europe/Athens",
    }
    a = requests.get(AIR_URL, params=a_params, timeout=30)
    a.raise_for_status()
    ca = a.json().get("current", {})

    rows.append({
        "city": name,
        "lat": lat,
        "lon": lon,
        "time": cw.get("time"),

        # weather
        "temp_c": cw.get("temperature_2m"),
        "humidity_pct": cw.get("relative_humidity_2m"),
        "wind_speed_ms": cw.get("wind_speed_10m"),
        "weather_code": cw.get("weather_code"),

        # air quality
        "aqi_eu": ca.get("european_aqi"),
        "pm25": ca.get("pm2_5"),
        "no2": ca.get("nitrogen_dioxide"),
        "o3": ca.get("ozone"),
    })

df = pd.DataFrame(rows)
display(df)


,city,lat,lon,time,temp_c,humidity_pct,wind_speed_ms,weather_code,aqi_eu,pm25,no2,o3
0,Lisbon,38.7167,-9.1333,2026-01-10T14:30,13.4,66,1.8,3,21,10.4,25.6,42.0
1,Thessaloniki,40.6436,22.9309,2026-01-10T14:30,10.0,84,4.1,3,26,5.6,8.9,60.0
2,Brussels,50.8505,4.3488,2026-01-10T14:30,-0.4,71,10.8,3,18,9.5,19.6,45.0
3,Barcelona,41.3888,2.1590,2026-01-10T14:30,12.3,47,15.3,2,25,2.4,6.8,63.0
4,Berlin,52.5244,13.4105,2026-01-10T14:30,-3.0,71,6.3,0,25,6.1,11.8,62.0


In [18]:
df.to_csv("raw_weather_air.csv", index=False)